### Install libs

In [75]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install -U opensearch-py==2.3.1 langchain==0.0.309 "pypdf>=3.8,<4" \
    apache-beam \
    datasets \
    tiktoken


  Obtaining dependency information for boto3>=1.28.57 from https://files.pythonhosted.org/packages/70/43/6f10af50c64e6184191a3fbda0889e9c9bc5c9224302cd41200968419e6d/boto3-1.29.2-py3-none-any.whl.metadata
  Obtaining dependency information for awscli>=1.29.57 from https://files.pythonhosted.org/packages/e2/ab/dd66c8153fd3659777c07e4c4d7ec3ce41df0a6de0ca1ce40043a401eb6a/awscli-1.30.2-py3-none-any.whl.metadata
  Obtaining dependency information for botocore>=1.31.57 from https://files.pythonhosted.org/packages/f4/a2/e03fbeaa13a25405ac6251d5deb10cd663416d7e45e46cf9e446a91b9e0c/botocore-1.32.2-py3-none-any.whl.metadata
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f8874a96c5061422625ba86c3ee1e6587ccd92ff9f5bf7bd91b2/s3transfer-0.7.0-py3-none-any.whl.metadata
  Using cached s3transfer-0.7.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached docutils-0.16-py2.py3

In [2]:
0

0

In [81]:
import warnings
warnings.filterwarnings('ignore')
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

os.environ["AWS_DEFAULT_REGION"] = "us-east-1" 

boto3_bedrock = bedrock.get_bedrock_client(
    #assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)


from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.load.dump import dumps

# - create the Anthropic Model
llm = Bedrock(
    model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 200}
)



Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


### Data preparation

In [102]:
from langchain.document_loaders import TextLoader
loader = TextLoader("./textdata/text_1.txt", encoding = 'UTF-8')

documents = loader.load()

In [84]:
documents

[Document(page_content='Công ty Chứng khoán của chúng tôi là công ty chứng khoán An Bình (Viết tắt: ABS) Địa chỉ: 36 Hoàng Cầu, Thành phố Hà Nội\nEmail: support@abs.vn\nSố Điện Thoại: (84) 123-456-789\nGiới thiệu:\nChúng tôi là một trong những công ty chứng khoán hàng đầu tại Việt Nam. Với hơn 20 năm kinh nghiệm trong lĩnh vực này, chúng tôi luôn cam kết mang đến cho khách hàng dịch vụ tốt nhất và giải pháp đầu tư tối ưu.\nDịch vụ chính:\n- Môi giới chứng khoán\n- Tư vấn đầu tư tài chính\n- Quản lý danh mục đầu tư\n- Dịch vụ cơ cấu tài sản\nSự kiện sắp diễn ra:\nCuộc họp cổ đông thường niên sẽ được tổ chức vào ngày 10/12/2023 tại khách sạn Bình Minh.\nCác nhân viên nổi bật:\n- Ông Nguyễn Văn A: Giám đốc điều hành\n- Bà Trần Thị B: Phó giám đốc\n- Anh Phạm C: Trưởng phòng Môi giới\nĐối tác chiến lược: Công ty Tài chính B, Ngân hàng C và Tổ chức Đầu tư D.\nMọi thông tin chi tiết về chúng tôi, vui lòng truy cập website chính thức: abs.vn Xu hướng giá cổ phiếu của Công ty Chứng khoán Beta 

In [103]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size= 1790 , #  because sbert 'max_seq_length': 256 ~ 70%*256 = 179 VN words -> *5 ->  895 chunk_size (characters)
    chunk_overlap= 150, # 70 ~ 14 words, one line.
)
docs = text_splitter.split_documents(documents)



In [106]:
docs1 = docs
docs1

[Document(page_content='Công ty Chứng khoán của chúng tôi là công ty chứng khoán An Bình (Viết tắt: ABS) Địa chỉ: 36 Hoàng Cầu, Thành phố Hà Nội\nEmail: support@abs.vn\nSố Điện Thoại: (84) 123-456-789\nGiới thiệu:\nChúng tôi là một trong những công ty chứng khoán hàng đầu tại Việt Nam. Với hơn 20 năm kinh nghiệm trong lĩnh vực này, chúng tôi luôn cam kết mang đến cho khách hàng dịch vụ tốt nhất và giải pháp đầu tư tối ưu.\nDịch vụ chính:\n- Môi giới chứng khoán\n- Tư vấn đầu tư tài chính\n- Quản lý danh mục đầu tư\n- Dịch vụ cơ cấu tài sản\nSự kiện sắp diễn ra:\nCuộc họp cổ đông thường niên sẽ được tổ chức vào ngày 10/12/2023 tại khách sạn Bình Minh.\nCác nhân viên nổi bật:\n- Ông Nguyễn Văn A: Giám đốc điều hành\n- Bà Trần Thị B: Phó giám đốc\n- Anh Phạm C: Trưởng phòng Môi giới\nĐối tác chiến lược: Công ty Tài chính B, Ngân hàng C và Tổ chức Đầu tư D.\nMọi thông tin chi tiết về chúng tôi, vui lòng truy cập website chính thức: abs.vn Xu hướng giá cổ phiếu của Công ty Chứng khoán Beta 

In [87]:
for doc in docs:
    print(doc)
    print("---"*10)

page_content='Công ty Chứng khoán của chúng tôi là công ty chứng khoán An Bình (Viết tắt: ABS) Địa chỉ: 36 Hoàng Cầu, Thành phố Hà Nội\nEmail: support@abs.vn\nSố Điện Thoại: (84) 123-456-789\nGiới thiệu:\nChúng tôi là một trong những công ty chứng khoán hàng đầu tại Việt Nam. Với hơn 20 năm kinh nghiệm trong lĩnh vực này, chúng tôi luôn cam kết mang đến cho khách hàng dịch vụ tốt nhất và giải pháp đầu tư tối ưu.\nDịch vụ chính:\n- Môi giới chứng khoán\n- Tư vấn đầu tư tài chính\n- Quản lý danh mục đầu tư\n- Dịch vụ cơ cấu tài sản\nSự kiện sắp diễn ra:\nCuộc họp cổ đông thường niên sẽ được tổ chức vào ngày 10/12/2023 tại khách sạn Bình Minh.\nCác nhân viên nổi bật:\n- Ông Nguyễn Văn A: Giám đốc điều hành\n- Bà Trần Thị B: Phó giám đốc\n- Anh Phạm C: Trưởng phòng Môi giới\nĐối tác chiến lược: Công ty Tài chính B, Ngân hàng C và Tổ chức Đầu tư D.\nMọi thông tin chi tiết về chúng tôi, vui lòng truy cập website chính thức: abs.vn Xu hướng giá cổ phiếu của Công ty Chứng khoán Beta trong thán

In [88]:
from langchain.document_loaders import TextLoader
loader2 = TextLoader("./textdata/text_2.txt", encoding = 'UTF-8')

documents2 = loader2.load()

In [91]:
text_splitter2 = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size= 1790 , #  because sbert 'max_seq_length': 256 ~ 70%*256 = 179 VN words -> *5 ->  895 chunk_size (characters)
    chunk_overlap= 150, #  70 ~ 14 words, one line.
)
docs2 = text_splitter2.split_documents(documents2)



In [92]:
for docs in docs2:
    print(docs)
    print("-"*50)

page_content='Giới thiệu chung:\nĐược thành lập từ năm 1993, cho đến nay Ngân hàng TMCP An Bình (ABBANK) đã và đang đóng góp cho thị trường tài chính Việt Nam một thương hiệu Ngân hàng bán lẻ thân thiện, uy tín và hiệu quả.\n\nCùng với sự phát triển uy tín thương hiệu, chất lượng môi trường làm việc tại ABBANK cũng liên tục được các tổ chức uy tín trong và ngoài nước đánh giá cao những năm gần đây: Năm 2021, ABBANK lọt vào danh sách 25 Thương hiệu Tài chính dẫn đầu tại Việt Nam theo công bố của Tạp chí Tài chính Forbes Việt Nam với giá trị thương hiệu ước đạt 39,4 triệu USD; ABBANK là 1 trong 10 ngân hàng ngoài quốc doanh có Chỉ số thương hiệu tốt nhất và Top 13 Ngân hàng có chỉ số thương hiệu tốt nhất (Theo báo cáo xếp hạng của Mibrand năm 2021); ABBANK cũng được tạp chí HR Asia chứng nhận là một trong những Nơi làm việc tốt nhất Châu Á kể từ năm 2020.\n\nVới nguồn lực tài chính vững mạnh, các đối tác chiến lược là các định chế tài chính lớn, các tập đoàn uy tín trên Thế giới và Việt 

In [109]:
docs1

[Document(page_content='Công ty Chứng khoán của chúng tôi là công ty chứng khoán An Bình (Viết tắt: ABS) Địa chỉ: 36 Hoàng Cầu, Thành phố Hà Nội\nEmail: support@abs.vn\nSố Điện Thoại: (84) 123-456-789\nGiới thiệu:\nChúng tôi là một trong những công ty chứng khoán hàng đầu tại Việt Nam. Với hơn 20 năm kinh nghiệm trong lĩnh vực này, chúng tôi luôn cam kết mang đến cho khách hàng dịch vụ tốt nhất và giải pháp đầu tư tối ưu.\nDịch vụ chính:\n- Môi giới chứng khoán\n- Tư vấn đầu tư tài chính\n- Quản lý danh mục đầu tư\n- Dịch vụ cơ cấu tài sản\nSự kiện sắp diễn ra:\nCuộc họp cổ đông thường niên sẽ được tổ chức vào ngày 10/12/2023 tại khách sạn Bình Minh.\nCác nhân viên nổi bật:\n- Ông Nguyễn Văn A: Giám đốc điều hành\n- Bà Trần Thị B: Phó giám đốc\n- Anh Phạm C: Trưởng phòng Môi giới\nĐối tác chiến lược: Công ty Tài chính B, Ngân hàng C và Tổ chức Đầu tư D.\nMọi thông tin chi tiết về chúng tôi, vui lòng truy cập website chính thức: abs.vn Xu hướng giá cổ phiếu của Công ty Chứng khoán Beta 

In [108]:
for doc in docs2:
    print(doc)
    print("-"*100)

page_content='Giới thiệu chung:\nĐược thành lập từ năm 1993, cho đến nay Ngân hàng TMCP An Bình (ABBANK) đã và đang đóng góp cho thị trường tài chính Việt Nam một thương hiệu Ngân hàng bán lẻ thân thiện, uy tín và hiệu quả.\n\nCùng với sự phát triển uy tín thương hiệu, chất lượng môi trường làm việc tại ABBANK cũng liên tục được các tổ chức uy tín trong và ngoài nước đánh giá cao những năm gần đây: Năm 2021, ABBANK lọt vào danh sách 25 Thương hiệu Tài chính dẫn đầu tại Việt Nam theo công bố của Tạp chí Tài chính Forbes Việt Nam với giá trị thương hiệu ước đạt 39,4 triệu USD; ABBANK là 1 trong 10 ngân hàng ngoài quốc doanh có Chỉ số thương hiệu tốt nhất và Top 13 Ngân hàng có chỉ số thương hiệu tốt nhất (Theo báo cáo xếp hạng của Mibrand năm 2021); ABBANK cũng được tạp chí HR Asia chứng nhận là một trong những Nơi làm việc tốt nhất Châu Á kể từ năm 2020.\n\nVới nguồn lực tài chính vững mạnh, các đối tác chiến lược là các định chế tài chính lớn, các tập đoàn uy tín trên Thế giới và Việt 

## Opensearch set up

In [93]:
import boto3
import time
vector_store_name = 'rag-pythonsql'
index_name = "infloatbase"

encryption_policy_name = "raql-encryp-policy"
network_policy_name = "raql-network-policy"
access_policy_name = 'raql-access-policy'
identity = boto3.client('sts').get_caller_identity()['Arn']

aoss_client = boto3.client('opensearchserverless')


## SageMaker embedding

In [94]:
## Todo: 
# Change endpoint_name to your Sagemaker sbert deployed model.

## Langchain Doc for SM Embedding Class Customization

from typing import Dict, List
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
import json


class ContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        """
        Transforms the input into bytes that can be consumed by SageMaker endpoint.
        Args:
            inputs: List of input strings.
            model_kwargs: Additional keyword arguments to be passed to the endpoint.
        Returns:
            The transformed bytes input.
        """
        # Example: inference.py expects a JSON string with a "inputs" key:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})  
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        """
        Transforms the bytes output from the endpoint into a list of embeddings.
        Args:
            output: The bytes output from SageMaker endpoint.
        Returns:
            The transformed output - list of embeddings
        Note:
            The length of the outer list is the number of input strings.
            The length of the inner lists is the embedding dimension.
        """
        # Example: inference.py returns a JSON string with the list of
        # embeddings in a "vectors" key:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["vectors"]


content_handler = ContentHandler()



infloat_batch_embeddings = SagemakerEndpointEmbeddings(
    # credentials_profile_name="credentials-profile-name",
    endpoint_name="huggingface-pytorch-inference-2023-11-17-04-03-54-290", # TODO: change this to your own infloat base endpoint
    region_name="us-east-1",  # TODO: change this to your sagemaker deployed sbert endpoint Region 
    content_handler=content_handler,
)

In [95]:
res_1 = infloat_batch_embeddings.embed_query("foo")
print(len(res_1)) # expect 768

res_2 = infloat_batch_embeddings.embed_documents(["foo"])
print(len(res_2) ) # expect 1
print(len(res_2[0]) ) # expect 768

768
1
768


In [20]:
res_1 = infloat_batch_embeddings.embed_query("foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo foo")


In [21]:
print(len(res_1))

768


In [23]:
sample_embedding = np.array(infloat_batch_embeddings.embed_query(docs[0].page_content))

# print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Size of the embedding:  (768,)


## Ingest to AOSS

In [96]:

security_policy = aoss_client.create_security_policy(
    name = encryption_policy_name,
    policy = json.dumps(
        {
            'Rules': [{'Resource': ['collection/' + vector_store_name],
            'ResourceType': 'collection'}],
            'AWSOwnedKey': True
        }),
    type = 'encryption'
)

network_policy = aoss_client.create_security_policy(
    name = network_policy_name,
    policy = json.dumps(
        [
            {'Rules': [{'Resource': ['collection/' + vector_store_name],
            'ResourceType': 'collection'}],
            'AllowFromPublic': True}
        ]),
    type = 'network'
)

collection = aoss_client.create_collection(name=vector_store_name,type='VECTORSEARCH')

while True:
    status = aoss_client.list_collections(collectionFilters={'name':vector_store_name})['collectionSummaries'][0]['status']
    if status in ('ACTIVE', 'FAILED'): break
    time.sleep(10)

access_policy = aoss_client.create_access_policy(
    name = access_policy_name,
    policy = json.dumps(
        [
            {
                'Rules': [
                    {
                        'Resource': ['collection/' + vector_store_name],
                        'Permission': [
                            'aoss:CreateCollectionItems',
                            'aoss:DeleteCollectionItems',
                            'aoss:UpdateCollectionItems',
                            'aoss:DescribeCollectionItems'],
                        'ResourceType': 'collection'
                    },
                    {
                        'Resource': ['index/' + vector_store_name + '/*'],
                        'Permission': [
                            'aoss:CreateIndex',
                            'aoss:DeleteIndex',
                            'aoss:UpdateIndex',
                            'aoss:DescribeIndex',
                            'aoss:ReadDocument',
                            'aoss:WriteDocument'],
                        'ResourceType': 'index'
                    }],
                'Principal': [identity],
                'Description': 'Easy data policy'}
        ]),
    type = 'data'
)

host = collection['createCollectionDetail']['id'] + '.' + os.environ.get("AWS_DEFAULT_REGION", None) + '.aoss.amazonaws.com:443'

In [97]:
host

'f4rkudrfg2b0fp2a8qhi.us-east-1.aoss.amazonaws.com:443'

In [110]:
docs1

[Document(page_content='Công ty Chứng khoán của chúng tôi là công ty chứng khoán An Bình (Viết tắt: ABS) Địa chỉ: 36 Hoàng Cầu, Thành phố Hà Nội\nEmail: support@abs.vn\nSố Điện Thoại: (84) 123-456-789\nGiới thiệu:\nChúng tôi là một trong những công ty chứng khoán hàng đầu tại Việt Nam. Với hơn 20 năm kinh nghiệm trong lĩnh vực này, chúng tôi luôn cam kết mang đến cho khách hàng dịch vụ tốt nhất và giải pháp đầu tư tối ưu.\nDịch vụ chính:\n- Môi giới chứng khoán\n- Tư vấn đầu tư tài chính\n- Quản lý danh mục đầu tư\n- Dịch vụ cơ cấu tài sản\nSự kiện sắp diễn ra:\nCuộc họp cổ đông thường niên sẽ được tổ chức vào ngày 10/12/2023 tại khách sạn Bình Minh.\nCác nhân viên nổi bật:\n- Ông Nguyễn Văn A: Giám đốc điều hành\n- Bà Trần Thị B: Phó giám đốc\n- Anh Phạm C: Trưởng phòng Môi giới\nĐối tác chiến lược: Công ty Tài chính B, Ngân hàng C và Tổ chức Đầu tư D.\nMọi thông tin chi tiết về chúng tôi, vui lòng truy cập website chính thức: abs.vn Xu hướng giá cổ phiếu của Công ty Chứng khoán Beta 

In [112]:
docs2

[Document(page_content='Giới thiệu chung:\nĐược thành lập từ năm 1993, cho đến nay Ngân hàng TMCP An Bình (ABBANK) đã và đang đóng góp cho thị trường tài chính Việt Nam một thương hiệu Ngân hàng bán lẻ thân thiện, uy tín và hiệu quả.\n\nCùng với sự phát triển uy tín thương hiệu, chất lượng môi trường làm việc tại ABBANK cũng liên tục được các tổ chức uy tín trong và ngoài nước đánh giá cao những năm gần đây: Năm 2021, ABBANK lọt vào danh sách 25 Thương hiệu Tài chính dẫn đầu tại Việt Nam theo công bố của Tạp chí Tài chính Forbes Việt Nam với giá trị thương hiệu ước đạt 39,4 triệu USD; ABBANK là 1 trong 10 ngân hàng ngoài quốc doanh có Chỉ số thương hiệu tốt nhất và Top 13 Ngân hàng có chỉ số thương hiệu tốt nhất (Theo báo cáo xếp hạng của Mibrand năm 2021); ABBANK cũng được tạp chí HR Asia chứng nhận là một trong những Nơi làm việc tốt nhất Châu Á kể từ năm 2020.\n\nVới nguồn lực tài chính vững mạnh, các đối tác chiến lược là các định chế tài chính lớn, các tập đoàn uy tín trên Thế giớ

In [113]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from langchain.vectorstores import OpenSearchVectorSearch

import langchain 


service = 'aoss'
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, "us-east-1",  service)

docsearch = OpenSearchVectorSearch.from_documents(
    docs2,
    infloat_batch_embeddings,
    bulk_size=1000,
    opensearch_url=host,
    http_auth=auth,
    timeout = 100,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    index_name=index_name,
    engine="faiss",
)

